# Hydra

In [ ]:
from functools import partial

import hydra
from hydra.utils import instantiate
from omegaconf import OmegaConf
import torch


class BoringModel(pl.LightningModule):

    def __init__(self, optim_partial, in_feats=4, out_feats=2):
        super().__init__()

        # Hydra config components
        self.optim_partial = optim_partial
        self.in_feats = in_feats
        self.out_feats = out_feats

        # Control weight randomness
        pl.seed_everything(1234)
        self.layer = torch.nn.Linear(in_feats, out_feats)

    def forward(self, x):
        return self.layer(x)

    def configure_optimizers(self):
        return self.optim_partial(self.parameters())


# Plain python approach
model = BoringModel(optim_partial=partial(torch.optim.Adam, lr=1e-5, weight_decay=0.2))
optimizer = model.configure_optimizers()

# Partial instantiate approach
OmegaConf.register_new_resolver("get_method", hydra.utils.get_method)
cfg = {
    '_target_': '__main__.BoringModel',
    'in_feats': 4,
    'out_feats': 2,
    'optim_partial': {
        '_target_': 'functools.partial',
        '_args_': ['${get_method: torch.optim.Adam}'],
        'lr': 1e-5,
        'weight_decay': 0.2}}
model2 = instantiate(cfg)
optimizer2 = model2.configure_optimizers()